In [4]:
"""
Challenge #2
영화 이름을 가지고 감독, 주요 출연진, 예산, 흥행 수익, 영화의 장르, 간단한 시놉시스 등 영화에 대한 정보로 답장하는 체인을 만드세요.
LLM은 항상 동일한 형식을 사용하여 응답해야 하며, 이를 위해서는 원하는 출력의 예시를 LLM에 제공해야 합니다.
예제를 제공하려면 FewShotPromptTemplate 또는 FewShotChatMessagePromptTemplate을 사용하세요.


Hint.
LLM이 답변 형식을 학습하도록 다양한 영화에 대한 예시를 만들어야 합니다.
예시는 과제의 요구조건을 만족시키려면 감독, 주요 출연진, 예산, 흥행 수익, 장르, 간략한 줄거리 가 포함되어야 합니다. LLM이 답변 형식을 효과적으로 학습하려면 모든 예시는 동일한 형식을 유지해야 합니다.
"""
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

#llm = OpenAI(model_name="gpt-3.5-turbo-1106")

llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature= 0.1,
    streaming=False,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

poem_prompt = ChatPromptTemplate.from_messages(
    [
        ("system",
         "너는 프로그래밍 언어에 대한 시를 쓰는 데 특화되어 있어."),
         ("human",
          "나는 {programming_language}에 관한 시를 원해."),
    ]
)

poem_chain = poem_prompt | llm

explain_prompt = ChatPromptTemplate.from_messages(
    [
        ("system","너는 시를 설명하는 데 특화되어있어."),
        ("human", "{poem}")
    ]
)

explain_chain = explain_prompt | llm

final_chain = {"poem": poem_prompt} | explain_chain

def generate_poem_and_explanation(programming_language):
    poem_result = poem_chain.invoke({"programming_language": programming_language})
    poem = str(poem_result.content) 

    explain_result = explain_chain.invoke({"poem": poem})
    explanation = str(explain_result.content)

    return poem, explanation

programming_language = "C++"
poem, explanation = generate_poem_and_explanation(programming_language)

print("### 시 ###")
print(poem)
print("\n### 설명 ###")
print(explanation)

### 시 ###
C++은 우아한 언어,
객체지향의 아름다움을 담은 언어.
포인터와 참조, 다양한 기능,
프로그래머의 상상을 현실로 만들어.

클래스와 상속, 다형성의 매력,
템플릿과 STL로 더욱 완벽해져.
메모리를 직접 다루는 힘,
C++은 프로그래머의 마음속에 살아있네.

### 설명 ###
이 시는 C++ 언어의 아름다움과 강점을 잘 담아내고 있네요. C++은 객체지향 프로그래밍을 지원하며 포인터, 참조, 클래스, 상속, 다형성, 템플릿, STL 등 다양한 기능을 제공하여 프로그래머가 더욱 다양하고 효율적인 프로그램을 작성할 수 있도록 도와줍니다. 또한 메모리를 직접 다룰 수 있는 강력한 기능을 제공하여 뛰어난 성능을 발휘할 수 있습니다. 이 시는 C++의 매력을 잘 표현한 것 같네요.
